# ２章　小売店のデータでデータ加工を行う

本章では、ある小売店の売上履歴と顧客台帳データを用いて、データ分析の素地となる「データの加工」を習得することが目的です。
実際の現場データは手入力のExcel等、決して綺麗なデータではない事が多いため、
データの揺れや整合性の担保など、汚いデータを取り扱うデータ加工を主体に進めて行きます。

### ノック１１：データを読み込んでみよう

In [178]:
import pandas as pd

uriage_data = pd.read_csv('uriage.csv')
uriage_data.head()

,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美
1,2019-07-13 13:05:29,商 品 S,NaN,浅田賢二
2,2019-05-11 19:42:07,商 品 a,NaN,南部慶二
3,2019-02-12 23:40:45,商品Z,2600.0,麻生莉緒
4,2019-04-22 03:09:35,商品a,NaN,平田鉄二


In [179]:
kokyaku_data = pd.read_excel('kokyaku_daicho.xlsx')
kokyaku_data.head()

,顧客名,かな,地域,メールアドレス,登録日
0,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018/01/04
1,岡田 敏也,おかだ としや,E市,okada_toshiya@example.com,42782
2,芳賀 希,はが のぞみ,A市,haga_nozomi@example.com,2018/01/07
3,荻野 愛,おぎの あい,F市,ogino_ai@example.com,42872
4,栗田 憲一,くりた けんいち,E市,kurita_kenichi@example.com,43127


### ノック１２：データの揺れを見てみよう

In [180]:
uriage_data['item_name'].head()

0      商品A
1    商 品 S
2    商 品 a
3      商品Z
4      商品a
Name: item_name, dtype: object

In [181]:
uriage_data['item_price'].head()

0     100.0
1       NaN
2       NaN
3    2600.0
4       NaN
Name: item_price, dtype: float64

### ノック１３：データに揺れがあるまま集計しよう

In [182]:
# 売上合計を月別で算出する
print(uriage_data['purchase_date'].dtypes)

object


In [183]:
# object型だからdatetime型に変換
print(pd.to_datetime(uriage_data['purchase_date']))
uriage_data['purchase_date'] = pd.to_datetime(uriage_data['purchase_date'])
print(uriage_data['purchase_date'].dtypes)

0      2019-06-13 18:02:34
1      2019-07-13 13:05:29
2      2019-05-11 19:42:07
3      2019-02-12 23:40:45
4      2019-04-22 03:09:35
               ...        
2994   2019-02-15 02:56:39
2995   2019-06-22 04:03:43
2996   2019-03-29 11:14:05
2997   2019-07-14 12:56:49
2998   2019-07-21 00:31:36
Name: purchase_date, Length: 2999, dtype: datetime64[ns]
datetime64[ns]


In [184]:
# 年月のデータを作成
print(uriage_data['purchase_date'].dt.strftime('%Y%m'))
uriage_data['purchase_month'] = uriage_data['purchase_date'].dt.strftime('%Y%m')

0       201906
1       201907
2       201905
3       201902
4       201904
         ...  
2994    201902
2995    201906
2996    201903
2997    201907
2998    201907
Name: purchase_date, Length: 2999, dtype: object


In [185]:
# 商品数を集計。クライアントは商品を26品取り扱っている。
# aggfunc='size' 各グループの要素数をカウントする
# fill_value=0 欠損値を0で埋める
res = uriage_data.pivot_table(index='purchase_month', columns= 'item_name', aggfunc='size', fill_value=0)
res

# 26品ではなく、99品ということがわかる。

item_name,商品W,商 品 n,商品E,商品M,商品P,商品S,商品W,商品X,商 品O,商 品Q,...,商品k,商品l,商品o,商品p,商品r,商品s,商品t,商品v,商品x,商品y
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,0,1,0,0,0,0,0,0,0,0,...,1,1,1,0,0,0,0,0,0,0
201902,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,1,1,0,0
201903,0,0,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201904,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
201905,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
201906,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
201907,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,2,0,0,0,0,0


In [186]:
# 商品金額を集計
res = uriage_data.pivot_table(index='purchase_month', columns= 'item_name', values= 'item_price',aggfunc='sum', fill_value=0)
res

# このままでは使い物にならない

item_name,商品W,商 品 n,商品E,商品M,商品P,商品S,商品W,商品X,商 品O,商 品Q,...,商品k,商品l,商品o,商品p,商品r,商品s,商品t,商品v,商品x,商品y
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,0.0,1400.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1100.0,1200.0,1500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
201902,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2400.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1900.0,2000.0,2200.0,0.0,0.0
201903,0.0,0.0,500.0,1300.0,1600.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
201904,2300.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1700.0,...,0.0,0.0,0.0,0.0,0.0,1900.0,0.0,0.0,0.0,0.0
201905,0.0,0.0,0.0,0.0,0.0,1900.0,0.0,0.0,0.0,0.0,...,0.0,1200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2500.0
201906,0.0,0.0,0.0,0.0,0.0,0.0,2300.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1600.0,0.0,0.0,0.0,0.0,2400.0,0.0
201907,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1500.0,0.0,1800.0,0.0,0.0,0.0,0.0,0.0


### ノック１４：商品名の揺れを補正しよう

In [187]:
# 現状把握のために現状の商品のユニーク数を算出
# pd.unique() で重複する要素を除く

print(len(pd.unique(uriage_data['item_name'])))

99


In [188]:
# データの揺れを補正処理
# .str.upper()は全ての文字を大文字に変換するが、元の文字を変換させるには変数に格納が必要。

uriage_data['item_name'] = uriage_data['item_name'].str.upper().str.replace(' ','').str.replace('　','')
uriage_data['item_name'].head()



0    商品A
1    商品S
2    商品A
3    商品Z
4    商品A
Name: item_name, dtype: object

In [189]:
# 商品名で昇順にソートする
uriage_data.sort_values(by=['item_name'], ascending=True)

,purchase_date,item_name,item_price,customer_name,purchase_month
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美,201906
1748,2019-05-19 20:22:22,商品A,100.0,松川綾女,201905
223,2019-06-25 08:13:20,商品A,100.0,板橋隆,201906
1742,2019-06-13 16:03:17,商品A,100.0,小平陽子,201906
1738,2019-02-10 00:28:43,商品A,100.0,松田浩正,201902
...,...,...,...,...,...
2880,2019-04-22 00:36:52,商品Y,NaN,田辺光洋,201904
2881,2019-04-30 14:21:09,商品Y,NaN,高原充則,201904
1525,2019-01-24 10:27:23,商品Y,2500.0,五十嵐春樹,201901
1361,2019-05-28 13:45:32,商品Y,2500.0,大崎ヒカル,201905


In [190]:
# データの補正結果検証
print(pd.unique(uriage_data['item_name']))
print(len(pd.unique(uriage_data['item_name'])))

['商品A' '商品S' '商品Z' '商品V' '商品O' '商品U' '商品L' '商品C' '商品I' '商品R' '商品X' '商品G'
 '商品P' '商品Q' '商品Y' '商品N' '商品W' '商品E' '商品K' '商品B' '商品F' '商品D' '商品M' '商品H'
 '商品T' '商品J']
26


### ノック１５：金額欠損値の補完をしよう

In [191]:
# データ全体の欠損値の存在を確認
#any()関数で行、または列に欠損値があるかどうかを判定してくれる。デフォルトではaxis = 0 で列を判定　axis = 1 で行を判定
uriage_data.isnull().any(axis=0)


purchase_date     False
item_name         False
item_price         True
customer_name     False
purchase_month    False
dtype: bool

欠損値を補完する処理

In [192]:
# 欠損値があるかどうかを示すブール値のシリーズ。isnull()はTrueが欠損を示す
# Trueの箇所が欠損値であることを示す
flg_is_null = uriage_data['item_price'].isnull()
flg_is_null.head()

0    False
1     True
2     True
3    False
4     True
Name: item_price, dtype: bool

In [193]:
# 欠損値を最大値で埋める

#.loc()は、条件を付与しそれに合致するデータの抽出ができる(ブーリアンインデックス)
# .loc(行を指定し、条件がTrueの値を選択, 条件を満たした行の列を選択)
#金額が欠損している商品名を取得し、trgに格納していく。
# .unique()は重複処理を避けるため
for trg in list(uriage_data.loc[flg_is_null, 'item_name'].unique()):

    # (~flg_is_null)はNOT演算子。0と1を反転できるので、欠損値がFalseとなる。
    # 金額が欠損していない商品名が、金額が欠損している商品名と合致した時、uriage_data['item_price']から取得する
    price = uriage_data.loc[(~flg_is_null) & (uriage_data['item_name'] == trg), 'item_price'].max()

    # 金額が欠損していない同じ商品から、欠損している金額に補完する
    uriage_data.loc[(flg_is_null) & (uriage_data['item_name'] == trg), 'item_price'] = price

uriage_data.head()

,purchase_date,item_name,item_price,customer_name,purchase_month
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美,201906
1,2019-07-13 13:05:29,商品S,1900.0,浅田賢二,201907
2,2019-05-11 19:42:07,商品A,100.0,南部慶二,201905
3,2019-02-12 23:40:45,商品Z,2600.0,麻生莉緒,201902
4,2019-04-22 03:09:35,商品A,100.0,平田鉄二,201904


In [194]:
# 改めてデータ全体の欠損値の存在を確認
#any()関数で行、または列に欠損値があるかどうかを判定してくれる。デフォルトではaxis = 0 で列を判定　axis = 1 で行を判定
uriage_data.isnull().any(axis=0)

purchase_date     False
item_name         False
item_price        False
customer_name     False
purchase_month    False
dtype: bool

In [195]:
# .min()のskipnaはデフォルトではTrue。NaNデータを無視する。skipna=FalseではNaNと記載。
for trg in list(uriage_data['item_name'].sort_values().unique()):
    print(trg + 'の最大額' + str(uriage_data.loc[uriage_data['item_name'] == trg]['item_price'].max())
        + trg + 'の最小額' + str(uriage_data.loc[uriage_data['item_name'] == trg]['item_price'].min(skipna = False))
        )


商品Aの最大額100.0商品Aの最小額100.0
商品Bの最大額200.0商品Bの最小額200.0
商品Cの最大額300.0商品Cの最小額300.0
商品Dの最大額400.0商品Dの最小額400.0
商品Eの最大額500.0商品Eの最小額500.0
商品Fの最大額600.0商品Fの最小額600.0
商品Gの最大額700.0商品Gの最小額700.0
商品Hの最大額800.0商品Hの最小額800.0
商品Iの最大額900.0商品Iの最小額900.0
商品Jの最大額1000.0商品Jの最小額1000.0
商品Kの最大額1100.0商品Kの最小額1100.0
商品Lの最大額1200.0商品Lの最小額1200.0
商品Mの最大額1300.0商品Mの最小額1300.0
商品Nの最大額1400.0商品Nの最小額1400.0
商品Oの最大額1500.0商品Oの最小額1500.0
商品Pの最大額1600.0商品Pの最小額1600.0
商品Qの最大額1700.0商品Qの最小額1700.0
商品Rの最大額1800.0商品Rの最小額1800.0
商品Sの最大額1900.0商品Sの最小額1900.0
商品Tの最大額2000.0商品Tの最小額2000.0
商品Uの最大額2100.0商品Uの最小額2100.0
商品Vの最大額2200.0商品Vの最小額2200.0
商品Wの最大額2300.0商品Wの最小額2300.0
商品Xの最大額2400.0商品Xの最小額2400.0
商品Yの最大額2500.0商品Yの最小額2500.0
商品Zの最大額2600.0商品Zの最小額2600.0


### ノック１６：顧客名の揺れを補正しよう

In [196]:
kokyaku_data.head()

,顧客名,かな,地域,メールアドレス,登録日
0,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018/01/04
1,岡田 敏也,おかだ としや,E市,okada_toshiya@example.com,42782
2,芳賀 希,はが のぞみ,A市,haga_nozomi@example.com,2018/01/07
3,荻野 愛,おぎの あい,F市,ogino_ai@example.com,42872
4,栗田 憲一,くりた けんいち,E市,kurita_kenichi@example.com,43127


In [197]:
uriage_data.head()

,purchase_date,item_name,item_price,customer_name,purchase_month
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美,201906
1,2019-07-13 13:05:29,商品S,1900.0,浅田賢二,201907
2,2019-05-11 19:42:07,商品A,100.0,南部慶二,201905
3,2019-02-12 23:40:45,商品Z,2600.0,麻生莉緒,201902
4,2019-04-22 03:09:35,商品A,100.0,平田鉄二,201904


In [198]:
kokyaku_data['顧客名'] = kokyaku_data['顧客名'].str.replace(' ','').str.replace('　','')
kokyaku_data['顧客名'].head()

0    須賀ひとみ
1     岡田敏也
2      芳賀希
3      荻野愛
4     栗田憲一
Name: 顧客名, dtype: object

### ノック１７：日付の揺れを補正しよう

In [199]:
kokyaku_data['登録日']

0      2018/01/04
1           42782
2      2018/01/07
3           42872
4           43127
          ...    
195    2017/06/20
196    2018/06/20
197    2017/04/29
198    2019/04/19
199    2019/04/23
Name: 登録日, Length: 200, dtype: object

In [200]:
# 日付のフォーマットが統一されていないので、補正処理
# .astype(str)で登録日を非文字列（数値）を文字列型に変換する。
# .isdigit()で数値のみで構成されているかを判定して　TrueかFalseを返す

# 非文字列データが含まれている場合、NaNが返される
print(kokyaku_data['登録日'].str.isdigit())
# 非文字列データを文字列型に変換する
print(kokyaku_data['登録日'].astype(str))

# 数値のみのデータの個数を把握
flg_is_serial = kokyaku_data['登録日'].astype(str).str.isdigit()
flg_is_serial.sum()

0      False
1        NaN
2      False
3        NaN
4        NaN
       ...  
195    False
196    False
197    False
198    False
199    False
Name: 登録日, Length: 200, dtype: object
0      2018/01/04
1           42782
2      2018/01/07
3           42872
4           43127
          ...    
195    2017/06/20
196    2018/06/20
197    2017/04/29
198    2019/04/19
199    2019/04/23
Name: 登録日, Length: 200, dtype: object


22

In [201]:
# 数値取り込まれている登録日を日付に補正する

# .loc()で判定した数値のみを抽出し、Excelなどのシリアル値をfloat型に変換し、それをpd.to_timedelta関数を使って時間差（timedelta）に変換。unit='D'で単位を日に指定。
# pd.to_timedeltaで算出した日数を基準値のpd.to_datetime('1900/01/01')に足し合わせて、実際の日付を計算する。
#fromSerial = pd.to_timedelta(kokyaku_data.loc[flg_is_serial, '登録日'].astype('float'), unit='D') + pd.to_datetime('1900/01/01')
print(pd.to_timedelta(kokyaku_data.loc[flg_is_serial, '登録日'].astype('float'), unit='D') + pd.to_datetime('1900/01/01'))


1     2017-02-18
3     2017-05-19
4     2018-01-29
21    2017-07-06
27    2017-06-17
47    2017-01-08
49    2017-07-15
53    2017-04-10
76    2018-03-31
80    2018-01-12
99    2017-06-01
114   2018-06-05
118   2018-01-31
122   2018-04-18
139   2017-05-27
143   2017-03-26
155   2017-01-21
172   2018-03-24
179   2017-01-10
183   2017-07-26
186   2018-07-15
192   2018-06-10
Name: 登録日, dtype: datetime64[ns]


In [202]:
# ExcelとPythonでは基準日からの数え方に違いがあるので2日分ずれていることがわかる。コードに-2を描き加えて処理する。
fromSerial = pd.to_timedelta(kokyaku_data.loc[flg_is_serial, '登録日'].astype('float') - 2, unit='D') + pd.to_datetime('1900/01/01')
print(fromSerial.dtypes)
fromSerial.head()

datetime64[ns]


1    2017-02-16
3    2017-05-17
4    2018-01-27
21   2017-07-04
27   2017-06-15
Name: 登録日, dtype: datetime64[ns]

In [203]:
# fromSerialは補正された日付のみ格納されている。正しい日付も書式を合わせる処理をする
#print(pd.to_datetime(kokyaku_data.loc[~flg_is_serial, '登録日']))

fromString = pd.to_datetime(kokyaku_data.loc[~flg_is_serial, '登録日'])
print(fromString.dtypes)

# /区切りが-区切りに統一された
fromString.head()

datetime64[ns]


0   2018-01-04
2   2018-01-07
5   2017-06-20
6   2018-06-11
7   2017-05-19
Name: 登録日, dtype: datetime64[ns]

In [204]:
# 数値に補正したデータ、書式を統一したデータを結合しデータを更新する
kokyaku_data['登録日'] = pd.concat([fromSerial, fromString])
kokyaku_data.head()

,顧客名,かな,地域,メールアドレス,登録日
0,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018-01-04
1,岡田敏也,おかだ としや,E市,okada_toshiya@example.com,2017-02-16
2,芳賀希,はが のぞみ,A市,haga_nozomi@example.com,2018-01-07
3,荻野愛,おぎの あい,F市,ogino_ai@example.com,2017-05-17
4,栗田憲一,くりた けんいち,E市,kurita_kenichi@example.com,2018-01-27


In [205]:
# 登録日から登録年月を算出する
print(kokyaku_data['登録日'].dt.strftime('%Y%m'))
kokyaku_data['登録年月'] = kokyaku_data['登録日'].dt.strftime('%Y%m')

rslt = kokyaku_data.groupby('登録年月').count()['顧客名']
print(sum(rslt))
print(len(kokyaku_data))

0      201801
1      201702
2      201801
3      201705
4      201801
        ...  
195    201706
196    201806
197    201704
198    201904
199    201904
Name: 登録日, Length: 200, dtype: object
200
200


In [206]:
# 登録日列に数値データが残っていないか判定

print(kokyaku_data['登録日'].astype(str).str.isdigit())
flg_is_serial = kokyaku_data['登録日'].astype(str).str.isdigit()
flg_is_serial.sum()

0      False
1      False
2      False
3      False
4      False
       ...  
195    False
196    False
197    False
198    False
199    False
Name: 登録日, Length: 200, dtype: bool


0

### ノック１８：顧客名をキーに２つのデータを結合(ジョイン)しよう

In [207]:
uriage_data.head(1)

,purchase_date,item_name,item_price,customer_name,purchase_month
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美,201906


In [208]:
kokyaku_data.head(1)

,顧客名,かな,地域,メールアドレス,登録日,登録年月
0,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018-01-04,201801


In [212]:
join_data = pd.merge(uriage_data, kokyaku_data,left_on = 'customer_name', right_on = '顧客名', how = 'left')
join_data = join_data.drop('customer_name', axis=1)
join_data.head()

,purchase_date,item_name,item_price,purchase_month,顧客名,かな,地域,メールアドレス,登録日,登録年月
0,2019-06-13 18:02:34,商品A,100.0,201906,深井菜々美,ふかい ななみ,C市,fukai_nanami@example.com,2017-01-26,201701
1,2019-07-13 13:05:29,商品S,1900.0,201907,浅田賢二,あさだ けんじ,C市,asada_kenji@example.com,2018-04-07,201804
2,2019-05-11 19:42:07,商品A,100.0,201905,南部慶二,なんぶ けいじ,A市,nannbu_keiji@example.com,2018-06-19,201806
3,2019-02-12 23:40:45,商品Z,2600.0,201902,麻生莉緒,あそう りお,D市,asou_rio@example.com,2018-07-22,201807
4,2019-04-22 03:09:35,商品A,100.0,201904,平田鉄二,ひらた てつじ,D市,hirata_tetsuji@example.com,2017-06-07,201706


### ノック１９：クレンジングしたデータをダンプしよう

In [213]:
# クレンジングしたデータのカラムを整理する
dump_data = join_data[['purchase_date', 'purchase_month', 'item_name', 'item_price', '顧客名', 'かな', '地域', 'メールアドレス', '登録日']]
dump_data.head()

,purchase_date,purchase_month,item_name,item_price,顧客名,かな,地域,メールアドレス,登録日
0,2019-06-13 18:02:34,201906,商品A,100.0,深井菜々美,ふかい ななみ,C市,fukai_nanami@example.com,2017-01-26
1,2019-07-13 13:05:29,201907,商品S,1900.0,浅田賢二,あさだ けんじ,C市,asada_kenji@example.com,2018-04-07
2,2019-05-11 19:42:07,201905,商品A,100.0,南部慶二,なんぶ けいじ,A市,nannbu_keiji@example.com,2018-06-19
3,2019-02-12 23:40:45,201902,商品Z,2600.0,麻生莉緒,あそう りお,D市,asou_rio@example.com,2018-07-22
4,2019-04-22 03:09:35,201904,商品A,100.0,平田鉄二,ひらた てつじ,D市,hirata_tetsuji@example.com,2017-06-07


In [214]:
# csvに出力,このデータを元に分析を行う。
dump_data.to_csv('dump_data.csv', index=False)

### ノック２０：データを集計しよう

In [215]:
import_data = pd.read_csv('dump_data.csv')
import_data.head()

,purchase_date,purchase_month,item_name,item_price,顧客名,かな,地域,メールアドレス,登録日
0,2019-06-13 18:02:34,201906,商品A,100.0,深井菜々美,ふかい ななみ,C市,fukai_nanami@example.com,2017-01-26
1,2019-07-13 13:05:29,201907,商品S,1900.0,浅田賢二,あさだ けんじ,C市,asada_kenji@example.com,2018-04-07
2,2019-05-11 19:42:07,201905,商品A,100.0,南部慶二,なんぶ けいじ,A市,nannbu_keiji@example.com,2018-06-19
3,2019-02-12 23:40:45,201902,商品Z,2600.0,麻生莉緒,あそう りお,D市,asou_rio@example.com,2018-07-22
4,2019-04-22 03:09:35,201904,商品A,100.0,平田鉄二,ひらた てつじ,D市,hirata_tetsuji@example.com,2017-06-07


In [216]:
# 購入年月、商品の集計
byItem = import_data.pivot_table(index='purchase_month', columns= 'item_name', aggfunc='size', fill_value=0)
byItem

item_name,商品A,商品B,商品C,商品D,商品E,商品F,商品G,商品H,商品I,商品J,...,商品Q,商品R,商品S,商品T,商品U,商品V,商品W,商品X,商品Y,商品Z
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,18,13,19,17,18,15,11,16,18,17,...,17,21,20,17,7,22,13,14,10,0
201902,19,14,26,21,16,14,14,17,12,14,...,22,22,22,23,19,22,24,16,11,1
201903,17,21,20,17,9,27,14,18,12,16,...,23,16,20,12,23,18,16,21,16,0
201904,17,19,24,20,18,17,14,11,18,13,...,20,20,16,16,11,15,14,16,20,0
201905,24,14,16,14,19,18,23,15,16,11,...,13,22,18,16,16,9,21,16,20,0
201906,24,12,11,19,13,18,15,13,19,22,...,15,16,21,12,18,20,17,15,13,0
201907,20,20,17,17,12,17,19,19,19,23,...,15,19,23,21,13,28,16,18,12,0


In [217]:
# 購入年月、購入金額の集計
byPrice = import_data.pivot_table(index='purchase_month', columns='item_name', values='item_price', aggfunc='sum', fill_value=0)
byPrice

item_name,商品A,商品B,商品C,商品D,商品E,商品F,商品G,商品H,商品I,商品J,...,商品Q,商品R,商品S,商品T,商品U,商品V,商品W,商品X,商品Y,商品Z
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,1800.0,2600.0,5700.0,6800.0,9000.0,9000.0,7700.0,12800.0,16200.0,17000.0,...,28900.0,37800.0,38000.0,34000.0,14700.0,48400.0,29900.0,33600.0,25000.0,0.0
201902,1900.0,2800.0,7800.0,8400.0,8000.0,8400.0,9800.0,13600.0,10800.0,14000.0,...,37400.0,39600.0,41800.0,46000.0,39900.0,48400.0,55200.0,38400.0,27500.0,2600.0
201903,1700.0,4200.0,6000.0,6800.0,4500.0,16200.0,9800.0,14400.0,10800.0,16000.0,...,39100.0,28800.0,38000.0,24000.0,48300.0,39600.0,36800.0,50400.0,40000.0,0.0
201904,1700.0,3800.0,7200.0,8000.0,9000.0,10200.0,9800.0,8800.0,16200.0,13000.0,...,34000.0,36000.0,30400.0,32000.0,23100.0,33000.0,32200.0,38400.0,50000.0,0.0
201905,2400.0,2800.0,4800.0,5600.0,9500.0,10800.0,16100.0,12000.0,14400.0,11000.0,...,22100.0,39600.0,34200.0,32000.0,33600.0,19800.0,48300.0,38400.0,50000.0,0.0
201906,2400.0,2400.0,3300.0,7600.0,6500.0,10800.0,10500.0,10400.0,17100.0,22000.0,...,25500.0,28800.0,39900.0,24000.0,37800.0,44000.0,39100.0,36000.0,32500.0,0.0
201907,2000.0,4000.0,5100.0,6800.0,6000.0,10200.0,13300.0,15200.0,17100.0,23000.0,...,25500.0,34200.0,43700.0,42000.0,27300.0,61600.0,36800.0,43200.0,30000.0,0.0


In [218]:
# 購入年月、購入顧客の集計
byCustomer = import_data.pivot_table(index='purchase_month', columns='顧客名', aggfunc='size', fill_value=0)
byCustomer

顧客名,さだ千佳子,中仁晶,中田美智子,丸山光臣,久保田倫子,亀井一徳,五十嵐春樹,井上桃子,井口寛治,井川真悠子,...,香椎優一,高原充則,高梨結衣,高沢美咲,高田さんま,鳥居広司,鶴岡薫,麻生莉緒,黄川田博之,黒谷長利
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,3,1,4,2,2,0,5,3,3,1,...,0,1,1,1,5,2,0,2,2,5
201902,9,1,2,2,1,4,2,1,0,4,...,4,0,3,2,0,1,2,4,0,1
201903,1,2,1,6,1,4,3,3,2,2,...,3,1,6,2,4,2,4,2,2,1
201904,0,3,1,2,0,2,2,0,3,2,...,2,4,2,3,4,3,2,1,2,0
201905,3,2,5,2,4,1,2,1,3,3,...,1,1,1,0,2,2,3,4,4,1
201906,1,3,0,4,1,1,1,2,2,3,...,7,3,0,2,1,0,2,1,2,4
201907,3,0,3,2,5,3,5,2,5,5,...,2,4,4,2,0,2,4,3,4,1


In [222]:
# 購入年月、購入地域の集計
byRegion = import_data.pivot_table(index='purchase_month', columns='地域', aggfunc='size', fill_value=0)
byRegion

地域,A市,B市,C市,D市,E市,F市,G市,H市
purchase_month,,,,,,,,
201901,59,55,72,34,49,57,49,42
201902,71,46,65,48,61,52,43,63
201903,64,52,57,43,52,59,51,59
201904,64,48,54,45,48,58,40,52
201905,57,52,68,48,59,65,35,43
201906,53,47,61,30,51,51,58,58
201907,76,53,61,42,54,64,47,54


In [225]:
#登録後に購入したことのない顧客の抽出
#顧客データにjoinして、購入日が欠損している顧客が該当客

away_data = pd.merge(uriage_data, kokyaku_data, left_on='customer_name', right_on='顧客名', how='right')
Customer_defection = away_data[away_data['purchase_date'].isnull()][['顧客名', 'メールアドレス', '登録日']]
Customer_defection

,顧客名,メールアドレス,登録日
2999,福井美希,fukui_miki1@example.com,2019-04-23
